In [127]:
from sqlalchemy import create_engine

In [128]:
username = 'dsbc_student'
password = '7*.8G9QH21'
host = '142.93.121.174'
port = '5432'
db = 'baseball'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(username, password,
                        host, port, db))

In [80]:
sql_people = '''
SELECT *
FROM people
LIMIT 20
'''

people = engine.execute(sql_people)
engine.dispose()
# people = people.fetchall()
# for row in people:
#     print(row)

sql_salaries = '''
SELECT *
FROM salaries
LIMIT 20
'''

salaries = engine.execute(sql_salaries)
engine.dispose()

sql_hof_inducted = '''
SELECT *
FROM hof_inducted
LIMIT 20
'''

hof_inducted = engine.execute(sql_hof_inducted)
engine.dispose()

sql_hof_not_inducted = '''
SELECT *
FROM hof_not_inducted
LIMIT 20
'''

hof_not_inducted = engine.execute(sql_hof_not_inducted)
engine.dispose()

In [5]:
people_rows = people.fetchall()
salaries_rows = salaries.fetchall()
hof_inducted_rows = hof_inducted.fetchall()
hof_not_inducted_rows = hof_not_inducted.fetchall()

1. Draw a schema of the relationship between the people, salaries, and hof_inducted tables. There are several online database schema drawers that you can use for free, including draw.io, which we recommend. We discuss the basics of how to use draw.io at the end of this checkpoint.
    1. Label the primary and foreign keys. (Note that a field can be a primary key to one table and a foreign key to another.)
    
    It looks like the playerid is the Parent/Foreign key.

    2. What are the parent and child tables? Are these one-to-one, one-to-many, or many-to-many relationships? 
    
    Just looking at the names of the tables, my guess is that the People table will be the parent table and the others will be child tables. I would think these would be one-to-one relationships, as each line represents a person and each person should only have one record (if any) per table. I think the hof_inducted may be a parent to the salaries table.

In [6]:
# print(people.keys())
# for row in people_rows:
#     print(row)
# print('\n')
# print(salaries.keys())
# for row in salaries_rows:
#     print(row)
# print('\n')
# print(hof_inducted.keys())
# for row in hof_inducted_rows:
#     print(row)
    
    
print('People Columns: ',people.keys())
print('\n')
print('Salaries Columns: ', salaries.keys())
print('\n')
print('hof_inducted Columns: ', hof_inducted.keys())
print('\n')
print('hof_not_inducted Columns: ', hof_not_inducted.keys())

People Columns:  ['playerid', 'birthyear', 'birthcountry', 'deathyear', 'namefirst', 'namelast', 'weight', 'height', 'bats', 'throws']


Salaries Columns:  ['yearid', 'teamid', 'lgid', 'playerid', 'salary']


hof_inducted Columns:  ['playerid', 'yearid', 'votedby', 'ballots', 'needed', 'votes', 'inducted', 'category', 'needed_note']


hof_not_inducted Columns:  ['playerid', 'yearid', 'votedby', 'ballots', 'needed', 'votes', 'inducted', 'category', 'needed_note']


2. Write a query that returns the namefirst and namelast fields of the people table, along with the inducted field from the hof_inducted table. All rows from the people table should be returned, and NULL values for the fields from hof_inducted should be returned when there is no match found.

I think this is correct, though it says None instead of Null.

In [7]:
sql = '''
SELECT namefirst, namelast, inducted
FROM people
LEFT OUTER JOIN hof_inducted ON people.playerid = hof_inducted.playerid
'''

inducted = engine.execute(sql)
engine.dispose()
inducted_rows = inducted.fetchall()

# for row in inducted_rows:
#     print(row)
    

3. In 2006, a special Baseball Hall of Fame induction was conducted for players from the negro baseball leagues of the 20th century. In that induction, 17 players were posthumously inducted into the Baseball Hall of Fame. Write a query that returns the first and last names, birth and death dates, and birth countries for these players. Note that the year of induction was 2006, and the value for votedby will be “Negro League.”

My query is showing 25 players instead of 17. My restraints are 'Negro League' in the votedby field and deathyear < 2006 as it is specified these players had passed at the time of induction. I added the inducted and votedby columns to QC the data and these columns are correct, so I'm not sure what's going on.

In [8]:
# negro_league_count = '''
# SELECT COUNT(votedby)
# FROM hof_inducted
# WHERE votedby = 'Negro League'
# '''

# negro_count = engine.execute(negro_league_count)
# engine.dispose()
# first_row = negro_count.first()
# count = first_row['count']
# print(count)

sql = '''
SELECT namefirst, namelast, birthyear, deathyear, birthcountry, inducted, votedby
FROM people
JOIN hof_inducted ON people.playerid = hof_inducted.playerid
WHERE votedby = 'Negro League' AND deathyear < 2000
'''

negro_league = engine.execute(sql)
engine.dispose()

# for row in negro_league:
#     print(row)

4. Write a query that returns the yearid, playerid, teamid, and salary fields from the salaries table, along with the category field from the hof_inducted table. Keep only the records that are in both salaries and hof_inducted. Hint: While a field named yearid is found in both tables, don’t JOIN by it. You must, however, explicitly name which field to include.

I think this is right - I believe INNER JOIN is the default for JOIN. I ran SELECT COUNT for both the salaries table and the new table and the salaries table has significantly more. What would be a good/best way to check this?

In [9]:
sql = '''
SELECT salaries.yearid, salaries.playerid, teamid, salary, category
FROM salaries
JOIN hof_inducted ON salaries.playerid = hof_inducted.playerid
'''

salaries_hof_inducted = engine.execute(sql)
engine.dispose()
print(salaries_hof_inducted.keys())
# for row in salaries_hof_inducted:
#     print(row)

['yearid', 'playerid', 'teamid', 'salary', 'category']


In [119]:
sql_people_count = '''
SELECT COUNT(*)
FROM people
'''

people_count = engine.execute(sql_people_count)
engine.dispose()
people_count = people_count.first()
people_count = people_count['count']
print('Number of records in People table: ', people_count)

sql_distinct_people_count = '''
SELECT COUNT(DISTINCT playerid)
FROM people
'''

people_distinct_count = engine.execute(sql_distinct_people_count)
engine.dispose()
people_distinct_count = people_distinct_count.first()
people_distinct_count = people_distinct_count['count']
print('Number of distinct records in People table: ', people_distinct_count)

sql_salaries_hof_count = '''
SELECT COUNT(*)
FROM salaries
JOIN hof_inducted ON salaries.playerid = hof_inducted.playerid
'''

salaries_hof_count = engine.execute(sql_salaries_hof_count)
engine.dispose()
first_row_sh_count = salaries_hof_count.first()
s_h_count = first_row_sh_count['count']
print('Number of records in Salaries hof_inducted join table: ', s_h_count)
    
sql_salaries_count = '''
SELECT COUNT(*)
FROM salaries
'''

salaries_count = engine.execute(sql_salaries_count)
engine.dispose()
first_row_sal_count = salaries_count.first()
s_count = first_row_sal_count['count']
print('Number of records in Salaries table: ', s_count)

sql_distinct_salaries_count = '''
SELECT COUNT(DISTINCT playerid)
FROM salaries
'''

salaries_distinct_count = engine.execute(sql_distinct_salaries_count)
engine.dispose()
first_row_distinct_sal_count = salaries_distinct_count.first()
s_distinct_count = first_row_distinct_sal_count['count']
print('Number of distinct records in Salaries table: ', s_distinct_count)

sql_hof_inducted_count = '''
SELECT COUNT(*)
FROM hof_inducted
'''

hof_inducted_count = engine.execute(sql_hof_inducted_count)
engine.dispose()
first_row_hof_count = hof_inducted_count.first()
hof_count = first_row_hof_count['count']
print('Number of records in hof_inducted table: ', hof_count)

sql_hof_not_inducted_count = '''
SELECT COUNT(*)
FROM hof_not_inducted
'''

hof_not_inducted_count = engine.execute(sql_hof_not_inducted_count)
engine.dispose()
first_row_not_hof_count = hof_not_inducted_count.first()
not_hof_count = first_row_not_hof_count['count']
print('Number of records in hof_not_inducted table: ', not_hof_count)

Number of records in People table:  19370
Number of distinct records in People table:  19370
Number of records in Salaries hof_inducted join table:  621
Number of records in Salaries table:  26428
Number of distinct records in Salaries table:  5149
Number of records in hof_inducted table:  323
Number of records in hof_not_inducted table:  3868


5. Write a query that returns the playerid, yearid, teamid, lgid, and salary fields from the salaries table and the inducted field from the hof_inducted table. Keep all records from both tables.

There are 26,428 records in the salaries table and 323 records in the hof_inducted table so I wouldn't expect to see more than 26,751 records in their union table, but 26,701 records seems a bit high - are there really 273 records that are not shared?

In [11]:
sql = '''
SELECT salaries.playerid, salaries.yearid, teamid, lgid, salary, inducted
FROM salaries
FULL OUTER JOIN hof_inducted ON salaries.playerid = hof_inducted.playerid
'''

salaries_hof = engine.execute(sql)
engine.dispose()
salaries_hof = salaries_hof.fetchall()
first_ten = salaries_hof[:10]

for row in first_ten:
    print(row)
    
sql_full_sal_hof = '''
SELECT COUNT(*)
FROM hof_inducted
FULL OUTER JOIN salaries ON salaries.playerid = hof_inducted.playerid
'''

sql_full_sal_hof = engine.execute(sql_full_sal_hof)
engine.dispose()
sql_full_sal_hof = sql_full_sal_hof.first()
sal_hof_count = sql_full_sal_hof['count']
print('Number of records in sal_hof_inducted table: ', sal_hof_count)

('barkele01', 1985, 'ATL', 'NL', 870000, None)
('bedrost01', 1985, 'ATL', 'NL', 550000, None)
('benedbr01', 1985, 'ATL', 'NL', 545000, None)
('campri01', 1985, 'ATL', 'NL', 633333, None)
('ceronri01', 1985, 'ATL', 'NL', 625000, None)
('chambch01', 1985, 'ATL', 'NL', 800000, None)
('dedmoje01', 1985, 'ATL', 'NL', 150000, None)
('forstte01', 1985, 'ATL', 'NL', 483333, None)
('garbege01', 1985, 'ATL', 'NL', 772000, None)
('harpete01', 1985, 'ATL', 'NL', 250000, None)
Number of records in sal_hof_inducted table:  26701


6. There are two tables, hof_inducted and hof_not_inducted, indicating successful and unsuccessful inductions into the Baseball Hall of Fame, respectively.

A. Combine these 2 tables by all fields. Keep all records.
Again, I believe this is correct, but I'm not sure the best way to check.

B. Get a distinct list of all player IDs for players who have been put up for HOF induction.
I had a lot of trouble with this and had to go to technical coaching. When I used the COUNT function to get the total number of records, I kept getting the count only for records in hof_inducted, and not both tables. The code that I thought would be correct is below.

I understand now that I needed to use DISTINCT on hof_not_inducated because it had duplicates, whereas all the values in hof_inducted were unique. What do you do if both tables have duplicate records?

SELECT COUNT(DISTINCT hof_inducted.playerid)
FROM hof_inducted
FULL OUTER JOIN hof_not_inducted ON hof_inducted.playerid = hof_not_inducted.playerid

In [12]:
sql_all = '''
SELECT *
FROM hof_inducted
FULL OUTER JOIN hof_not_inducted ON hof_inducted.playerid = hof_not_inducted.playerid
'''

hof_not_hof = engine.execute(sql_all)
engine.dispose()
hof_not_hof = hof_not_hof.fetchall()
hof_not_hof = hof_not_hof[:7]
for row in hof_not_hof:
    print(row)


sql_nominated = '''
SELECT DISTINCT(hof_inducted.playerid)
FROM hof_inducted
FULL OUTER JOIN hof_not_inducted ON hof_inducted.playerid = hof_not_inducted.playerid
'''

nominated = engine.execute(sql_nominated)
engine.dispose()
nominated = nominated.fetchall()
nominated = nominated[:20]
for row in nominated:
    print(row)
    
sql_count_nominated = '''
SELECT COUNT(DISTINCT(hof_not_inducted.playerid))
FROM hof_not_inducted
FULL OUTER JOIN hof_inducted ON hof_inducted.playerid = hof_not_inducted.playerid
'''

count_nominated = engine.execute(sql_count_nominated)
engine.dispose()
count_nominated = count_nominated.first()
count_nominated = count_nominated[0]
print(count_nominated)

('lajoina01', 1937, 'BBWAA', 201, 151, 168, 'Y', 'Player', None, 'lajoina01', 1936, 'BBWAA', 226, 170, 146, 'N', 'Player', None)
('speaktr01', 1937, 'BBWAA', 201, 151, 165, 'Y', 'Player', None, 'speaktr01', 1936, 'BBWAA', 226, 170, 133, 'N', 'Player', None)
('youngcy01', 1937, 'BBWAA', 201, 151, 153, 'Y', 'Player', None, 'youngcy01', 1936, 'BBWAA', 226, 170, 111, 'N', 'Player', None)
('hornsro01', 1942, 'BBWAA', 233, 175, 182, 'Y', 'Player', None, 'hornsro01', 1936, 'BBWAA', 226, 170, 105, 'N', 'Player', None)
('cochrmi01', 1947, 'BBWAA', 161, 121, 128, 'Y', 'Player', None, 'cochrmi01', 1936, 'BBWAA', 226, 170, 80, 'N', 'Player', None)
('sislege01', 1939, 'BBWAA', 274, 206, 235, 'Y', 'Player', None, 'sislege01', 1936, 'BBWAA', 226, 170, 77, 'N', 'Player', None)
('collied01', 1939, 'BBWAA', 274, 206, 213, 'Y', 'Player', None, 'collied01', 1936, 'BBWAA', 226, 170, 60, 'N', 'Player', None)
('crawfsa01',)
(None,)
('manleef99',)
('willivi01',)
('hunteca01',)
('gehrich01',)
('dandrra99',)
('

In [13]:
sql_hof_inducted_count_player = '''
SELECT COUNT(DISTINCT playerid)
FROM hof_inducted
'''

hof_inducted_count = engine.execute(sql_hof_inducted_count_player)
engine.dispose()
first_row_hof_count = hof_inducted_count.first()
hof_count = first_row_hof_count['count']
print('Number of distinct player records in hof_inducted table: ', hof_count)

sql_hof_not_inducted_count = '''
SELECT COUNT(DISTINCT playerid)
FROM hof_not_inducted
'''

hof_not_inducted_count = engine.execute(sql_hof_not_inducted_count)
engine.dispose()
first_row_not_hof_count = hof_not_inducted_count.first()
not_hof_count = first_row_not_hof_count['count']
print('Number of distinct player records in hof_not_inducted table: ', not_hof_count)

Number of distinct player records in hof_inducted table:  323
Number of distinct player records in hof_not_inducted table:  1140


7. Write a query that returns the last name, first name (see people table), and total recorded salaries for all players found in the salaries table.

I think this is right!

In [14]:
sql = '''
SELECT namelast, namefirst, salary
FROM salaries
LEFT JOIN people ON salaries.playerid = people.playerid
'''

name_salaries = engine.execute(sql)
engine.dispose()
name_salaries = name_salaries.fetchall()
name_salaries = name_salaries[:10]
for row in name_salaries:
    print(row)

('Barker', 'Len', 870000)
('Bedrosian', 'Steve', 550000)
('Benedict', 'Bruce', 545000)
('Camp', 'Rick', 633333)
('Cerone', 'Rick', 625000)
('Chambliss', 'Chris', 800000)
('Dedmon', 'Jeff', 150000)
('Forster', 'Terry', 483333)
('Garber', 'Gene', 772000)
('Harper', 'Terry', 250000)


8. Write a query that returns all records from the hof_inducted and hof_not_inducted tables that include playerid, yearid, namefirst, and namelast. Hint: Each FROM statement will include a LEFT OUTER JOIN!

Why wouldn't you just use FULL OUTER JOIN?

In [15]:
sql = '''
SELECT hof_inducted.playerid, yearid, namefist, namelast
FROM hof_inducted
FULL OUTER JOIN hof_not_inducted ON hof_inducted.playerid = hof_not_inducted.playerid
'''

9. Return a table including all records from both hof_inducted and hof_not_inducted, and include a new field, namefull, which is formatted as namelast , namefirst (in other words, the last name, followed by a comma, then a space, then the first name). The query should also return the yearid and inducted fields. Include only records since 1980 from both tables. Sort the resulting table by yearid, then inducted so that Y comes before N. Finally, sort by the namefull field, A to Z.

This was very tricky - I first forgot that simply joining two tables at a column wouldn't automatically select that column, so I kept trying to create the namefull column without using the people table which obviously didn't work. Then I had trouble with the WHERE statement - where to put it (under all the joins?) and which table to pull the column from (does it matter?). There are lots of multiples - is this because we haven't used DISTINCT? There are 17 Rick Aguileras...that doesn't seem right...

In [35]:
sql = '''
SELECT CONCAT(people.namelast, ', ', people.namefirst) AS namefull, hof_inducted.yearid, hof_inducted.inducted
FROM hof_inducted

FULL OUTER JOIN hof_not_inducted 
ON hof_inducted.yearid = hof_not_inducted.yearid

JOIN people
ON hof_not_inducted.playerid = people.playerid

WHERE hof_inducted.yearid > 1980
ORDER BY namefull
'''

namefull = engine.execute(sql)
engine.dispose()
namefull = namefull.fetchall()
namefull = namefull[:10]
for row in namefull:
    print(row)

('Abbott, Jim', 2005, 'Y')
('Abbott, Jim', 2005, 'Y')
('Aguilera, Rick', 2006, 'Y')
('Aguilera, Rick', 2006, 'Y')
('Aguilera, Rick', 2006, 'Y')
('Aguilera, Rick', 2006, 'Y')
('Aguilera, Rick', 2006, 'Y')
('Aguilera, Rick', 2006, 'Y')
('Aguilera, Rick', 2006, 'Y')
('Aguilera, Rick', 2006, 'Y')


10. Write a query that returns the highest annual salary for each teamid, ranked from high to low, along with the corresponding playerid. Bonus! Return namelast and namefirst in the resulting table. (You can find these in the people table.)

In [151]:
# print('My \' query:')
# '''SELECT CONCAT(namelast, ',',  namefirst) AS namefull, MAX(salary) AS max_salary, teamid, people.playerid
# FROM people

# FULL OUTER JOIN salaries
# ON salaries.playerid = people.playerid

# GROUP BY teamid, namefull, teamid, people.playerid

# ORDER BY max_salary DESC
# '''

# max_sal_each_team = engine.execute(max_sal_each_team_sql)
# engine.dispose()
# max_sal_each_team = max_sal_each_team.fetchall()
# max_sal_each_team = max_sal_each_team[-10:]
# for row in max_sal_each_team:
#     print(row)

print('Yunus\' query:')
max_sal_each_team_sql = '''
SELECT CONCAT(p.namelast, ',', p.namefirst) AS namefull, p.playerid, s.teamid, new_table.max_salary
FROM people AS p
INNER JOIN salaries AS s ON p.playerid = s.playerid
INNER JOIN (SELECT MAX(salary) AS max_salary, teamid FROM people
FULL OUTER JOIN salaries
ON salaries.playerid = people.playerid
GROUP BY teamid) AS new_table ON s.teamid = new_table.teamid AND s.salary = new_table.max_salary
ORDER BY new_table.max_salary DESC
'''

max_sal_each_team = engine.execute(max_sal_each_team_sql)
engine.dispose()
max_sal_each_team = max_sal_each_team.fetchall()
max_sal_each_team = max_sal_each_team[-10:]
for row in max_sal_each_team:
    print(row)

max_sal_sql = '''
SELECT MAX(new_table.max_salary)
FROM (SELECT CONCAT(namelast, ',', namefirst) AS namefull, MAX(salary) AS max_salary, teamid, people.playerid
FROM people
FULL OUTER JOIN salaries
ON salaries.playerid = people.playerid
GROUP BY teamid, namefull, teamid, people.playerid
ORDER BY max_salary DESC) AS new_table
'''

max_sal = engine.execute(max_sal_sql)
engine.dispose()
max_sal = max_sal.first()
max_sal = max_sal['max']
print('Max Salary: ', max_sal)

# max_sal = max_sal[-10:]
# for row in max_sal:
#     print(row)
    
    
# count = '''
# SELECT COUNT(teamid), MAX(salary)
# FROM salaries

# FULL OUTER JOIN people
# ON salaries.playerid = people.playerid
# '''

# count = engine.execute(count)
# engine.dispose()
# #count = count.fetchall(
# count = count.first()
# count = count['count']
# print(count)
    
# max_sal_sql = '''
# SELECT teamid, MAX(salary)
# FROM salaries
# GROUP BY teamid

# ORDER BY MAX(salary) DESC
# '''

# max_sal_only = engine.execute(max_sal_sql)
# engine.dispose()
# max_sal_only = max_sal_only.fetchall()
# max_sal_only = max_sal_only[:10]
# for row in max_sal_only:
#     print(row)

Yunus' query:
('Swisher,Nick', 'swishni01', 'CLE', 15000000)
('Lowe,Derek', 'lowede01', 'CLE', 15000000)
('Sheffield,Gary', 'sheffga01', 'FLO', 14936667)
('Price,David', 'priceda01', 'TBA', 14000000)
('Holliday,Matt', 'hollima01', 'OAK', 13500000)
('Shields,James', 'shielja02', 'KCA', 13500000)
('Vaughn,Mo', 'vaughmo01', 'ANA', 13166667)
('Guerrero,Vladimir', 'guerrvl01', 'MON', 11500000)
('Vaughn,Greg', 'vaughgr01', 'ML4', 5875000)
('Finley,Chuck', 'finlech01', 'CAL', 5375000)
Max Salary:  33000000


11. Select birthyear, deathyear, namefirst, and namelast of all the players born since the birth year of Babe Ruth (playerid = ruthba01). Sort the results by birth year from low to high.

I need to get better at subqueries and understanding which information to use to obtain what I need.

In [63]:
sql = '''
SELECT birthyear, deathyear, namefirst, namelast
FROM people
WHERE birthyear > 

(SELECT birthyear
FROM people
WHERE playerid = 'ruthba01')

ORDER BY birthyear
'''

birthyears = engine.execute(sql)
engine.dispose()
birthyears = birthyears.fetchall()
birthyears = birthyears[:10]
for row in birthyears:
    print(row)

(1896, 1978, 'Mike', 'Wilson')
(1896, 1983, 'Chick', 'Sorrells')
(1896, 1963, 'Merito', 'Acosta')
(1896, 1962, 'Mutt', 'Wilson')
(1896, 1950, 'Dick', 'McCabe')
(1896, 1969, 'Roy', 'Wilson')
(1896, 1969, 'Ray', 'Schmandt')
(1896, 1972, 'Rip', 'Conway')
(1896, 1968, 'Bill', 'Sherdel')
(1896, 1971, 'Sam', 'Post')


12. Using the people table, write a query that returns namefirst, namelast, and a field called usaborn where. The usaborn field should show the following: if the player's birthcountry is the USA, then the record is 'USA.' Otherwise, it's 'non-USA.' Order the results by 'non-USA' records first.

In [77]:
sql = '''
SELECT namefirst, namelast,
CASE
    WHEN birthcountry = 'USA' THEN 'USA'
    ELSE 'non-USA'
    END AS usaborn
FROM people
ORDER BY CASE
    WHEN birthcountry = 'USA' THEN 'USA'
    ELSE 'non-USA'
    END
'''

usaborn = engine.execute(sql)
engine.dispose()
usaborn = usaborn.fetchall()
# for row in usaborn:
#     print(row)


13. Calculate the average height for players throwing with their right hand versus their left hand. Name these fields right_height and left_height, respectively.

If the values are in inches, this looks like it could be right...

In [88]:
sql = '''
SELECT
AVG(CASE WHEN throws = 'R' THEN height END) AS right_height, 
AVG(CASE WHEN throws = 'L' THEN height END) AS left_height
FROM people
'''

handed = engine.execute(sql)
engine.dispose()
handed = handed.fetchall()
for row in handed:
    print(row)

(Decimal('72.3172024506092104'), Decimal('72.5530219780219780'))


14. Get the average of each team's maximum player salary since 2010. Hint: WHERE will go inside your CTE.

Along with the questions using cases and subqueries, I had to use my notes - I could not create these queries from memory.

In [98]:
sql = '''
WITH avg_max_sal AS
(SELECT teamid, MAX(salary) AS max_salary
FROM salaries
GROUP BY teamid
)

SELECT AVG(max_salary)
FROM avg_max_sal
'''

avg_max_sal = engine.execute(sql)
engine.dispose()
avg_max_sal = avg_max_sal.fetchall()
for row in avg_max_sal:
    print(row)

(Decimal('20483789.542857142857'),)
